In [1]:
%load_ext lab_black

In [2]:
import os
import time
import numpy as np
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta

In [3]:
import getpass

token = getpass.getpass("token:")
pro = ts.pro_api(token)

token: ························································


# 获取当天日期
```python
Parameters: None

Returns: str ('20190325')
```

In [4]:
get_date = lambda: time.strftime("%Y%m%d", time.localtime(time.time()))

# 获取前N天日期
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    N: int 前 N 天，默认 1 天

Returns: str ('20190324')
```

In [5]:
get_pastdate = lambda date=get_date(), N=1: datetime.strftime(
    pd.to_datetime(date) - timedelta(N + 1 - 1), "%Y%m%d"
)

# 获取N天后日期
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    N: int 后 N 天，默认 1 天

Returns: str ('20190324')
```

In [6]:
get_afterdate = lambda date=get_date(), N=1: datetime.strftime(
    pd.to_datetime(date) + timedelta(N + 1 - 1), "%Y%m%d"
)

# 获取最近N天单只个股的数据
```python
Parameters:
    
    code: str ('000001.SZ') 没有默认
        
    date: str ('20190325') 默认今天
        
    peroid: int 从 date 往前几天，默认 300 天

Returns: DataFrame 该支股票的历史数据
```

In [7]:
def get_oneNdays(code, date=get_date(), period=300):
    start_date = get_pastdate(date=date, N=period)
    name = pro.stock_basic(exchange="", list_status="L", fields="ts_code,name")
    df = pro.daily(ts_code=code, start_date=start_date, end_date=date).iloc[::-1]
    df["state"] = df["close"] >= df["open"]  # 阳柱:True, 阴柱:False
    df.index = range(len(df))
    df.name = "%s-%s" % (name[name["ts_code"] == code]["name"].item(), code)
    return df

# 凹口线

```python
判断凹口的函数，在 plotly_ohlc 中当输入为 draw_line={'凹口':} 时调用，因此不建议单独运行，也不设默认值。

前后 min_range 天的低点，往前 back_days 内找一天高点，当天涨幅 ('close'-'open') < increase，前后 max_range 天的 max('open','close') 不能超过当天的 max('open','close')。
    
Parameters:
    
    df: get_oneNdays()
        
    min_range: 找最低点时考虑前后各几项
        
    back_days: 从低点往前找的天数上限
        
    increase: 当天涨幅必须达到的标准
        
    max_range: 找最高点时考虑前后各几项

Returns: dict 每个 key 是一个 index（整数），也就是凹口线的起点，每个 value 是一个凹口线的值，也就是 max('open','close')
```

In [8]:
def aokou(df, min_range, back_days, increase, max_range):
    df["min"] = (
        df["close"].rolling(2 * min_range + 1, min_periods=min_range, center=True).min()
    )
    bottom = []
    for i in range(len(df) - 1, 0, -1):
        if df.loc[i]["close"] == df.loc[i]["min"]:
            bottom.append(i)

    df["max"] = (
        df[["close", "open"]]
        .max(axis=1)
        .rolling(2 * max_range + 1, min_periods=max_range, center=True)
        .max()
    )
    top = []
    for i in range(len(df) - 1, 0, -1):
        if max(df.loc[i]["open"], df.loc[i]["close"]) == df.loc[i]["max"]:
            top.append(i)

    x, y = [], []
    bottom_list = []
    for btm in bottom:
        while top:
            tp = top.pop(0)
            if (
                tp < btm
                and (df.loc[tp]["close"] - df.loc[tp]["open"])
                < df.loc[tp]["open"] * increase
                and df.loc[tp]["max"] not in y
            ):
                if len(x) == 0:
                    x.append(tp)
                    y.append(df.loc[tp]["max"])
                    bottom_list.append(btm)
                    break
                elif x[-1] - tp > min_range:
                    x.append(tp)
                    y.append(df.loc[tp]["max"])
                    bottom_list.append(btm)
                    break
                else:
                    pass
    return [dict(zip(x, y)), dict(zip(x, bottom_list))]

# 黄金线

```python
判断黄金线（黄金柱柱顶）的函数，在 plotly_ohlc 中当输入为 draw_line={'黄金':} 时调用，因此不建议单独运行，也不设默认值。
    
Parameters:
    
    df: get_oneNdays()

Returns: Returns: dict 每个 key 是一个 index（整数），每个 value 是一个黄金线的值，也就是 max('open','close')
```

In [9]:
def huangjin(df):
    x, y = [], []
    for i in range(len(df) - 2):
        o1 = df.loc[i]["open"]
        c1, c2, c3 = (
            df.loc[i]["close"],
            df.loc[i + 1]["close"],
            df.loc[i + 2]["close"],
        )
        v1, v2, v3 = (df.loc[i]["vol"], df.loc[i + 1]["vol"], df.loc[i + 2]["vol"])
        cond_open = o1 < min(c2, c3)  # 收盘价三日不破底
        cond_close = c1 < min(c2, c3)  # 收盘价三日不破顶
        cond_vol = (v1 > v2) & (v2 > v3)  # 量柱群三日不过头、不抬头
        if cond_open and cond_close and cond_vol:
            x.append(i)
            y.append(max(df.loc[i]["open"], df.loc[i]["close"]))
    return dict(zip(x, y))

# 倍量线

```python
判断倍量线（倍量柱柱底）的函数，在 plotly_ohlc 中当输入为 draw_line={'倍量':} 时调用，因此不建议单独运行，也不设默认值。
    
Parameters:
    
    df: get_oneNdays()
        
    v_pct_min: float 今天的量比上昨天的量的最小值
        
    v_pct_max: float 今天的量比上昨天的量的最大值
        
    c_pct_min: float 今天的收盘价比上昨天的收盘价的最小值

Returns: Returns: dict 每个 key 是一个 index（整数），每个 value 是一个倍量线的值，也就是 min('open','close')
```

In [10]:
def beiliang(df, v_pct_min, v_pct_max, c_pct_min):
    x, y = [], []
    for i in range(len(df) - 1):
        yesterday_v = df.loc[i]["vol"]
        yesterday_c = df.loc[i]["close"]
        today_v = df.loc[i + 1]["vol"]
        today_c = df.loc[i + 1]["close"]
        v_pct = today_v / yesterday_v
        c_pct = today_c / yesterday_c
        cond1 = v_pct > v_pct_min and v_pct < v_pct_max
        cond2 = c_pct > c_pct_min
        if cond1 and cond2:
            x.append(i + 1)
            y.append(min(df.loc[i + 1]["open"], df.loc[i + 1]["close"]))
    return dict(zip(x, y))

# 买入信号
```python
突破凹口线 max("open","close")>min(ak_line)，前提是这条凹口线的起点必须在150天以内，而且往前 100 天内要有 1 次以上倍量
```


In [32]:
def buy(df, i, new_ak, bl, print_mode=False):
    global ak_line, ak
    if len(ak_line) > 0 and df.loc[i, "high"] > min(ak_line):  # 如果当天在凹口里面，而且突破凹口线
        break_line = max(
            set(ak_line) - set([j for j in ak_line if j > df.loc[i, "high"]])
        )  # 可能一次突破2条凹口线，所以取突破的最大凹口线
        new_ak_line = [j for j in ak_line if j > df.loc[i, "high"]]  # 突破的凹口线就把它去掉
        l = list(ak[0].keys()).copy()
        for m in l:
            if m < i and ak[0][m] < df.loc[i, "high"]:
                ak[0].pop(m)
                ak[1].pop(m)
        ak_line = new_ak_line
        start = max(0, i - 100)
        if (
            len([b for b in bl.keys() if b < i and b > start]) > 1
        ):  # 如果从起点到现在这段时间里面黄金柱的数量大于1
            if print_mode:
                print(
                    df.loc[i, "trade_date"],
                    "最高价:",
                    "%5.2f" % df.loc[i, "high"],
                    "突破:",
                    "%5.2f" % break_line,
                    "凹口线起点:",
                    df.loc[
                        list(new_ak.keys())[list(new_ak.values()).index(break_line)],
                        "trade_date",
                    ],
                    "倍量柱数量:",
                    len([b for b in bl.keys() if b < i and b > start]),
                )
            # 返回当天的 index 和由持仓线构成的 list，因为之后要持仓线要变动，所以采用 list 保存历史信息，其中原始平仓线设在买入线下面1%
            return break_line
        else:
            return False
    else:
        return False

# 卖出信号
跌破持仓线（持续增加中的最大值）就卖出
1. 开盘卖
2. 盘中卖

In [48]:
def sell(df, i, print_mode=False):
    global line
    if df.loc[i, "open"] < max(line):  # 当天开盘就跌破持仓线（持续增加中的最大值）
        if print_mode:
            print(
                df.loc[i, "trade_date"],
                "开盘卖:",
                "%5.2f" % df.loc[i, "open"],
                "持仓:",
                "%5.2f" % max(line),
            )
        line = []
        return df.loc[i, "open"]  # 以开盘价卖出
    elif df.loc[i, "low"] < max(line):  # 当天盘中跌破持仓线（持续增加中的最大值）
        sold = max(line)
        if print_mode:
            print(
                df.loc[i, "trade_date"], "盘中卖:", "%5.2f" % sold, "持仓:", "%5.2f" % sold
            )
        line = []
        return sold  # 以持仓线（持续增加中的最大值）卖出
    else:
        return False

# 平仓线调整
1. 上涨超过20%，每上涨20%设置check点，设置新平仓线为check点与买入点的中间价格
2. 当上涨超过50%，平仓线设置于3/4位点

In [46]:
def adj(df, i, line):
    if (df.loc[i, "high"] - line[0]) / line[0] > 0.5:
        line.append(line[0] + (df.loc[i, "high"] - line[0]) * 0.75)
    elif (df.loc[i, "high"] - line[0]) / line[0] > 0.2:
        line.append((df.loc[i, "high"] + line[0]) * 0.5)
    return line

# 凹口线更新

```python
不能用到未来的信息，所以必须等到最低点后 7 天（ 7 天后才知道是最低点）才能画出凹口线，并且凹口线的起点必须要在最近 150 天以内才有效。
```

In [40]:
def update_ak_line(i, ak):
    ak_index = []
    for j in ak[0].keys():
        if i - j < 150 and i > ak[1][j] + 7:
            ak_index.append(j)
    return dict(zip(ak_index, [ak[0][x] for x in ak_index]))

# BackTest
```python
因为前面混乱的代码使用了全局变量，所以这部分的主代码不能包装成函数orz，如果要迭代，就写进 for loop

print_detail = False 每次买卖的信息

print_return = False 一支股票交易一年后的总年化收益率

手续费: 买卖时收取千分之一的手续费

```

In [75]:
print_detail = True
print_return = True
return_list = []
for code in ["600783.SH", "600888.SH", "600666.SH", "600218.SH"]:
    if print_detail:
        print(code)
    period = (
        pd.to_datetime(get_date()) - pd.to_datetime(get_pastdate(get_date(), 365))
    ).days
    df = get_oneNdays(code, date=get_date(), period=period)

    ak = aokou(df, min_range=7, back_days=365, increase=0.09, max_range=7)
    bl = beiliang(df, v_pct_min=1.9, v_pct_max=2.1, c_pct_min=1.01)

    line = []
    ass = []
    ass_return = 1
    poss = False
    for i in range(len(df)):
        new_ak = update_ak_line(i, ak)
        ak_line = list(new_ak.values())
        if not poss:
            temp = buy(df, i, new_ak, bl, print_mode=print_detail)
            if temp != False:
                poss = True
                ass.append(temp)
                line.append(temp * 0.99)
        else:
            line = adj(df, i, line)
            temp = sell(df, i, print_mode=print_detail)
            if temp != False:
                poss = False  # 空仓
                ass.append(temp)
                ass_return *= ass[-1] * 0.999 / ass[-2] * 1.001
        if i == len(df) - 1 and poss:  # 最后一天还没空仓
            ass.append(df.loc[i, "open"])
            ass_return *= ass[-1] * 0.999 / ass[-2] * 1.001
            if print_detail:
                print(df.loc[i, "trade_date"], "最后一天开盘卖:", "%5.2f" % df.loc[i, "open"])
    return_list.append(ass_return - 1)
    if print_return:
        print(code, "收益率", "%5.2f" % (ass_return - 1))

600783.SH
20181105 最高价: 10.78 突破: 10.29 凹口线起点: 20180725 倍量柱数量: 2
20181109 开盘卖: 14.15 持仓: 14.40
20181112 最高价: 15.54 突破: 11.81 凹口线起点: 20180521 倍量柱数量: 3
20181121 盘中卖: 21.30 持仓: 21.30
20190226 最高价: 25.06 突破: 23.44 凹口线起点: 20181120 倍量柱数量: 2
20190301 开盘卖: 21.08 持仓: 23.21
600783.SH 收益率  1.23
600888.SH
20190321 最高价:  6.38 突破:  6.00 凹口线起点: 20180809 倍量柱数量: 2
20190322 开盘卖:  5.91 持仓:  5.94
600888.SH 收益率 -0.02
600666.SH
20190219 最高价:  3.72 突破:  3.67 凹口线起点: 20181212 倍量柱数量: 2
20190220 盘中卖:  3.63 持仓:  3.63
20190225 最高价:  3.83 突破:  3.81 凹口线起点: 20181122 倍量柱数量: 2
20190226 盘中卖:  3.77 持仓:  3.77
20190304 最高价:  3.98 突破:  3.88 凹口线起点: 20180921 倍量柱数量: 2
20190305 开盘卖:  3.80 持仓:  3.84
20190306 最高价:  4.31 突破:  4.10 凹口线起点: 20180910 倍量柱数量: 2
20190311 开盘卖:  4.00 持仓:  4.06
600666.SH 收益率 -0.06
600218.SH
20190114 最高价:  5.54 突破:  5.49 凹口线起点: 20180925 倍量柱数量: 2
20190115 开盘卖:  5.43 持仓:  5.44
20190116 最高价:  6.10 突破:  5.62 凹口线起点: 20180910 倍量柱数量: 2
20190118 盘中卖:  6.15 持仓:  6.15
20190220 最高价: 12.06 突破: 10.05 凹口线起点: 20190128 倍量柱数

In [76]:
return_list

[1.2301864699107559,
 -0.015000985000000133,
 -0.06352398730139708,
 0.42104212558665943]